In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
# Set the database path to a location with write permissions
db_path = '../race_league_results.db'

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


# How to rank...
1. Determine each racer's percentile for every race
2. Get a median percentile for the year
3. Ranking is based on your last 3 years -> take median from last year, or the median before that, or 3 years past. If non exist, use the all time median. Otherwise, Null...

```py
df['_percentile'] = df.groupby('race_id')['_time'].rank(pct=True, ascending=True)
    
    # Calculating median percentile rank for each racer per year
    median_percentiles = df.groupby(['racer_id', 'year'])['_percentile'].median().reset_index()
```

In [3]:
def compute_rank(row, cols, fallback_col):
    """
    row: the row from the DataFrame
    cols: list of columns to consider (e.g. [cols[-1], cols[-2], cols[-3]])
    fallback_col: the fallback column name if all three are NaN (e.g. 'all_time_avg_percentile')
    """
    # Example weights
    w1, w2, w3 = 0.3, 0.5, 0.2
    
    values = [row[cols[0]], row[cols[1]], row[cols[2]]]
    weights = [w1, w2, w3]
    
    # Filter only non-null values and their corresponding weights
    valid_pairs = [(val, wt) for val, wt in zip(values, weights) if pd.notnull(val)]
    
    if not valid_pairs:
        # If all are null, return the fallback value
        return row[fallback_col]
    else:
        # Weighted average over the non-null columns
        total_weight = sum(pair[1] for pair in valid_pairs)
        weighted_sum = sum(pair[0] * pair[1] for pair in valid_pairs)
        return weighted_sum / total_weight

# Function to calculate aggregated and last 3 years race result percentiles for a given dataset
def calculate_aggregated_percentiles(df):
    # Calculating percentile ranks for each race
    # Lower times are better, so we use ascending=True
    df['percentile'] = df.groupby('race_id')['best_time'].rank(pct=True, ascending=True)
    df['position'] = df.groupby('race_id')['best_time'].rank(method='min', ascending=True).astype(int)
    
    # Calculating median percentile rank for each racer per year
    median_percentiles = df.groupby(['racer_id', 'year'])['percentile'].median().reset_index()
    
    # Creating a dictionary to hold pre-season and post-season rankings for each year
    rankings = {}
    
    # Iterating over each year present in the dataset
    for year in sorted(median_percentiles['year'].unique()):
        # Getting the median percentiles for the current year
        current_year_rankings = median_percentiles[median_percentiles['year'] == year]
    
        # Sorting by median percentile (better performance has lower percentile)
        current_year_rankings = current_year_rankings.sort_values('percentile')
    
        # Assigning post-season rankings based on sorted order
        current_year_rankings['post_season_rank'] = range(1, len(current_year_rankings) + 1)
    
        # Assigning pre-season rankings
        if year - 1 in rankings:
            # If the previous year exists, use its post-season rankings
            previous_year_rankings = rankings[year - 1][['racer_id', 'post_season_rank']]
            current_year_rankings = current_year_rankings.merge(previous_year_rankings, on='racer_id', how='left', suffixes=('', '_prev'))
            current_year_rankings['pre_season_rank'] = current_year_rankings['post_season_rank_prev'].fillna(max(current_year_rankings['post_season_rank']) + 1)
            current_year_rankings.drop(columns=['post_season_rank_prev'], inplace=True)
        else:
            # If no previous year data, assign a default pre-season rank
            current_year_rankings['pre_season_rank'] = max(current_year_rankings['post_season_rank']) + 1
    
        # Adding to the rankings dictionary
        rankings[year] = current_year_rankings
    
    # Combining all years' rankings into a single DataFrame
    all_years_rankings = pd.concat(rankings.values())
    
    # Calculating all-time percentile ranking for each racer: TODO, update this!
    all_time_percentiles = df.groupby('racer_id')['percentile'].mean().reset_index()
    all_time_percentiles.rename(columns={'percentile': 'all_time_avg_percentile'}, inplace=True)

    # Getting the last 3 years
    last_3_years = df['year'].unique()[-3:]
    columns = []

    # Initializing a DataFrame to hold last 3 years' percentiles
    last_3_years_percentiles = pd.DataFrame()

    # Iterating over each of the last 3 years
    for year in last_3_years:
        # Calculating median percentile for each racer for the given year
        year_percentiles = df[df['year'] == year].groupby('racer_id')['percentile'].median().reset_index()
        year_percentiles.rename(columns={'percentile': f'median_percentile_{year}'}, inplace=True)
        columns.append(f'median_percentile_{year}')

        # Merging with the main DataFrame
        if last_3_years_percentiles.empty:
            last_3_years_percentiles = year_percentiles
        else:
            last_3_years_percentiles = last_3_years_percentiles.merge(year_percentiles, on='racer_id', how='outer')

    # Merging all-time percentiles with last 3 years' percentiles
    aggregated_df = all_time_percentiles.merge(last_3_years_percentiles, on='racer_id', how='left')

    ## Finally, create a ranking from the last 3 years and avg data:    
    #aggregated_df['rank'] = aggregated_df.apply(
    #    lambda x: x[columns[-1]] if pd.notnull(x[columns[-1]]) else x[columns[-2]] if pd.notnull(x[columns[-2]]) else x[columns[-3]] if pd.notnull(x[columns[-3]]) else x['all_time_avg_percentile'],
    #    axis=1
    #)
    
    # Example usage in your apply:
    aggregated_df['rank'] = aggregated_df.apply(
        lambda x: compute_rank(
            row=x,
            cols=[columns[-1], columns[-2], columns[-3]],  # or however you define your three columns
            fallback_col='all_time_avg_percentile'
        ),
        axis=1
    )

    return aggregated_df, all_years_rankings

def last_5_races(df):
    df = df.sort_values(["racer_id", "race_date"])
    last_5 = df.groupby("racer_id", group_keys=False).tail(5)
    stats = (
        last_5.groupby("racer_id")["position"]
              .agg(avg_position_last5=("mean"), median_position_last5=("median"))
              .reset_index()
    )
    return stats

need a df with: racer_id, best_time, race_id, race_date (year?)

Make sure to filter out any race where the racer did not finish! -> i.e., best time < 9998

In [6]:
sql = """
SELECT results.race_id, results.racer_id, results.discipline, results.best_time, races.race_date, races.year
FROM (
    SELECT racer_id, discipline, best_time, race_id
    FROM RaceResults
    WHERE COALESCE(best_time, 9999) < 9998
) AS results
JOIN (
    SELECT race_id, race_date, substr(description, 1,4) as year
    FROM Races
) AS races
ON results.race_id = races.race_id
"""

df_combined = pd.read_sql_query(sql, conn)
df_combined['year'] = df_combined['year'].astype(int)

In [7]:
#mikemctaggart -> michaelmctaggart
df_combined.loc[df_combined['racer_id'] == 'mikemctaggart', 'racer_id'] = 'michaelmctaggart'

In [8]:
df_combined.head(2)

,race_id,racer_id,discipline,best_time,race_date,year
0,1,jeffcox,SKI,35.82,2014-01-01 00:00:00,2014
1,1,mcleanwood,SKI,36.10,2014-01-01 00:00:00,2014


In [9]:
ski_df = df_combined[df_combined['discipline'] == 'SKI']
snbd_df = df_combined[df_combined['discipline'] == 'SNBD']

In [10]:
ski_aggregated_df, ski_all_time = calculate_aggregated_percentiles(ski_df)
snbd_aggregated_df, snbd_all_time = calculate_aggregated_percentiles(snbd_df)

/var/folders/cp/tk2rjzwj4n76t_7mcr1f2y640000gn/T/ipykernel_21895/2388674575.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['percentile'] = df.groupby('race_id')['best_time'].rank(pct=True, ascending=True)
/var/folders/cp/tk2rjzwj4n76t_7mcr1f2y640000gn/T/ipykernel_21895/2388674575.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['position'] = df.groupby('race_id')['best_time'].rank(method='min', ascending=True).astype(int)
/var/folders/cp/tk2rjzwj4n76t_7mcr1f2y640000gn/T/ipykernel_21895/23886

In [11]:
ski_aggregated_df.sort_values(by='rank', ascending=True, inplace=True)

In [12]:
ski_aggregated_df.head(5)

,racer_id,all_time_avg_percentile,median_percentile_2023,median_percentile_2024,median_percentile_2025,rank
244,michaelmctaggart,0.026884,0.017857,0.027397,0.02381,0.024413
261,nickparr,0.024698,NaN,NaN,NaN,0.024698
51,caleweinberg,0.025832,NaN,NaN,NaN,0.025832
180,jordankofman,0.031356,NaN,NaN,NaN,0.031356
350,willstewart,0.041388,NaN,0.033673,NaN,0.033673


# Join to start lists, and append last 5 race results!

In [13]:
from helper_functions import clean_string

In [14]:
ski_start_list = pd.read_csv("2025/SKI_startlist.csv")
snbd_start_list = pd.read_csv("2025/SNBD_startlist.csv")

ski_start_list["racer_id"] = ski_start_list["Name"].apply(clean_string)
snbd_start_list["racer_id"] = snbd_start_list["Name"].apply(clean_string)

In [15]:
ski_df_last5 = last_5_races(ski_df)
snbd_df_last5 = last_5_races(snbd_df)

In [16]:
joined_ski = ski_start_list.merge(ski_aggregated_df, how="left", on="racer_id")
joined_ski = joined_ski.merge(ski_df_last5, how="left", on="racer_id")

joined_ski.sort_values(by='rank', ascending=True, inplace=True)
joined_ski

,Name,racer_id,all_time_avg_percentile,median_percentile_2023,median_percentile_2024,median_percentile_2025,rank,avg_position_last5,median_position_last5
34,Michael McTaggart,michaelmctaggart,0.026884,0.017857,0.027397,0.023810,0.024413,1.60,1.0
71,Will Stewart,willstewart,0.041388,NaN,0.033673,NaN,0.033673,2.75,2.0
58,Nicholas Balan,nicholasbalan,0.056637,NaN,0.022856,0.083333,0.045535,2.20,2.0
20,Brandon Hune,brandonhune,0.084002,0.066741,0.058006,0.047619,0.056637,3.20,3.0
26,Mclean Wood,mcleanwood,0.069453,0.051724,0.089483,NaN,0.078695,4.20,3.0
...,...,...,...,...,...,...,...,...,...
42,JD Etter,jdetter,0.976190,NaN,NaN,0.976190,0.976190,41.00,41.0
63,Dave Graff,davegraff,1.000000,NaN,NaN,1.000000,1.000000,42.00,42.0
50,Tori Johnston,torijohnston,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,Jeremy Ramshaw,jeremyramshaw,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
joined_snbd = snbd_start_list.merge(snbd_aggregated_df, how="left", on="racer_id")
joined_snbd = joined_snbd.merge(snbd_df_last5, how="left", on="racer_id")

joined_snbd.sort_values(by='rank', ascending=True, inplace=True)
joined_snbd

,Name,racer_id,all_time_avg_percentile,median_percentile_2022,median_percentile_2024,median_percentile_2025,rank,avg_position_last5,median_position_last5
3,Joel Farber,joelfarber,0.249447,NaN,0.187135,NaN,0.187135,3.600000,4.0
6,Thomas Daly,thomasdaly,0.333333,NaN,NaN,0.333333,0.333333,1.000000,1.0
5,Graham Ramshaw,grahamramshaw,0.377412,NaN,0.391667,NaN,0.391667,7.250000,7.5
2,Bernard Oegema,bernardoegema,0.303167,NaN,0.406433,NaN,0.406433,6.200000,7.0
8,Mark Sandell,marksandell,0.542982,NaN,0.578947,NaN,0.578947,10.666667,11.0
1,Steve Crawford,stevecrawford,0.672398,NaN,0.647661,0.666667,0.654788,10.800000,12.0
0,Sheri Ramshaw,sheriramshaw,0.686349,NaN,0.729532,NaN,0.729532,12.200000,13.0
4,Kyla Taylor,kylataylor,0.978216,NaN,0.948684,1.000000,0.967928,15.400000,18.0
7,Jacob Max Farber,jacobmaxfarber,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Save outputs!

In [22]:
joined_ski.to_csv("2025/ski_stats_2025.csv", index=False)
joined_snbd.to_csv("2025/snbd_stats_2025.csv", index=False)

# Appendix: looking up racer data

In [19]:
sql = """
SELECT *
FROM RaceResults
WHERE racer_id like '%farber%'
and racer_id like '%jacob%'
;
"""

foo2 = pd.read_sql_query(sql, conn)
foo2

,racer_id,discipline,team,tier,run1,run2,best_time,points,race_id,bib
0,jacobmaxfarber,SNBD,None,None,9999.0,9999.0,9999.0,None,241229,109
